In [ ]:
import requests
import requests_cache
from bs4 import BeautifulSoup
import time
import json
import re
import pandas as pd # Dùng để đọc và hiển thị file csv/tsv
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from requests_html import HTML

________________________________________

In [ ]:
# Lưu user
users = set()
# Lưu track
tracks = {"track":[], "user":[]}
# Lưu playlist
playlists = {"playlist":[], "user":[], "tracks":[]}
# Lưu link playlist của producer
urls_user_playlist = set()
# Lưu link playlist của discover
urls_playlist = []

**Chuyển từ list sang csv/tsv**

In [ ]:
def list_to_tsv_csv(listItem, outputFile):
    df = pd.DataFrame(listItem)
    df.to_csv(outputFile, sep='\t')

**Bấm để đồng ý cookies**

In [ ]:
def accept_cookies():
    soup = BeautifulSoup(browser.page_source)
    cookies = browser.find_element(By.XPATH,"//button[@id='onetrust-accept-btn-handler']")
    cookies.click()

**Bấm vào các nút > đến hết ở trang discover**

In [ ]:
def click_button():
    soup = BeautifulSoup(browser.page_source)
    genres = soup.find_all("li", "mixedModularHome__item")

    for i in range(len(genres)):
        lists = genres[i].find_all("div", "tileGallery__sliderPanelSlide")
        j = 4
        while True:
            if j > len(lists):
                break
            browser.find_elements(By.XPATH,"//button[contains(.,'forward')]")[i].click()
            j += 4

**Lấy hết tất cả playlists ở trang discover**

In [ ]:
def get_all_playlists_discover():
    soup = BeautifulSoup(browser.page_source)  
    genres = soup.find_all("li", "mixedModularHome__item")

    for i in range(len(genres)):
        lists = genres[i].find_all("div", "tileGallery__sliderPanelSlide")
        for pl in lists:
            title = pl.find("a", "playableTile__heading").get_text().strip()
            user = pl.find("a", "playableTile__usernameHeading").get_text().strip()
            link = pl.find("a", "playableTile__artworkLink")["href"]
            if title not in playlists:
                playlists["playlist"].append(title)
                playlists["user"].append(user)
                urls_playlist.append("https://soundcloud.com" + link)

**Kéo xuống đến hết trang**

In [ ]:
def scroll_down():
    soup = BeautifulSoup(browser.page_source)
    
    count = int(soup.find("div", "genericTrackCount__title").get_text())
    
    i = 15
    while i < count:
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight)")
        time.sleep(3)
        i += 15

**Lấy tất cả users và tracks của một playlist**

In [ ]:
def get_all_users_tracks_playlist():
    scroll_down()
    
    soup = BeautifulSoup(browser.page_source)
    search = soup.find_all("li", "systemPlaylistTrackList__item")
    if len(search) == 0:
        search = soup.find_all("li", "trackList__item")

    # Trích xuất user và track trong 1 playlist
    trackIds = []
    for producer in search:
        user = producer.find("a", "trackItem__username").get_text()
        url = producer.find("a", "trackItem__username").get('href')
        track = producer.find("a", "trackItem__trackTitle").get_text()
        
        users.add(user)
        urls_user_playlist.add("https://soundcloud.com" + url + "/sets")
        
        if track not in tracks["track"]:
            tracks["track"].append(track)
            tracks["user"].append(user)
        trackIds.append(tracks["track"].index(track))
        
    playlists["tracks"].append(trackIds)

**Lấy tất cả playlists của một user**

In [ ]:
def get_all_playlists_user(browser):
    soup = BeautifulSoup(browser.page_source)
    posts = soup.find_all("li", "soundList__item")
    
    # kiểm tra có playlist hay không, nếu có chạy dưới, nếu không bỏ qua
    if len(posts) != 0:
        # nếu số playlists lớn hơn 4 thì sẽ scroll
        if len(posts) > 4:
            scroll_down2()
            soup = BeautifulSoup(browser.page_source)
            
        # trích xuất playlist
        for post in posts:
            # lấy tên playlist
            title = post.find("a", "soundTitle__title").get_text().strip()
            playlists["playlist"].append(title)
            
            # lấy user của playlist
            user = post.find("a", "soundTitle__usernameText").get_text().strip()
            playlists["user"].append(user)

            # lấy Id của các tracks có trong playlist theo index
            songs = post.find_all("li", "compactTrackList__item")
            # lưu index của track
            trackIds = []
            for song in songs:
                trackTitle = song.get_text().strip()
                # nếu có track mới
                if trackTitle not in tracks:
                    tracks["track"].append(trackTitle)
                # lấy index của track
                trackIds.append(tracks["track"].index(trackTitle))
            playlists["user"].append(trackIds)

In [ ]:
def parse_users_tracks_per_genre():
    for url in urls_playlist:
        browser.execute_script('window.open("{link}");'.format(link = url));
        browser.switch_to.window(browser.window_handles[1])
        time.sleep(2)
        get_all_users_tracks_playlist()
        time.sleep(1)
        browser.close()
        browser.switch_to.window(browser.window_handles[0])

In [ ]:
def parse_playlists_per_genre():
    for url in urls_user_playlist:
        browser.execute_script('window.open("{link}");'.format(link = url));
        browser.switch_to.window(browser.window_handles[1])
        time.sleep(2)
        get_all_playlists_user()
        time.sleep(1)
        browser.close()
        browser.switch_to.window(browser.window_handles[0])

In [ ]:
browser = webdriver.Chrome()
browser.get("https://soundcloud.com/discover")

accept_cookies()

click_button()
get_all_playlists_discover()

parse_users_tracks_per_genre()

browser.quit()

In [ ]:
# nên check lại bởi vì vài lần chưa scroll hết :<
def scroll_down2():
    soup = BeautifulSoup(browser.page_source)
    
    for i in range(4):
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight)")
        time.sleep(3)

In [ ]:
browser = webdriver.Chrome()
browser.get("https://soundcloud.com/discover")

time.sleep(5)
accept_cookies()

parse_playlists_per_genre()

**Lưu data vào file tsv**

In [ ]:
outputFile = "Crawl_data/user.csv"
list_to_tsv_csv(users, outputFile)
outputFile = "Crawl_data/track.csv"
list_to_tsv_csv(tracks, outputFile)
outputFile = "Crawl_data/playlist.csv"
list_to_tsv_csv(playlists, outputFile)

In [ ]:
print(len(users))
print(len(tracks["user"]))
print(len(playlists))